<a href="https://colab.research.google.com/github/aijurist/Sexual-harassment-Classification---DL/blob/main/MultiLabel_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import tensorflow as tf
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import string

#Text preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss
from sklearn.metrics import multilabel_confusion_matrix,classification_report, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, auc

import warnings
warnings.filterwarnings('ignore')

import pickle #Saving model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = "".join([c for c in text if c not in string.punctuation])
    tokens = [token for token in text.split() if token.lower() not in stop_words]
    processed_text = ' '.join(tokens)
    print(processed_text)
    return processed_text


df = pd.read_csv('/content/drive/My Drive/sexual_harassment.csv')
df = df.dropna()
df['Description'] = df['Description'].apply(preprocess_text)

#Tokenizing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Description'])
sequences = tokenizer.texts_to_sequences(df['Description'])
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post')

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df[['Commenting', 'Ogling/Facial Expressions/Staring', 'Touching /Groping']], test_size=0.2, random_state=42)


Streaming output truncated to the last 5000 lines.
man road near maratha mandir telling quoti love babyquot girls passing
crossing roadsit raining quite heavily man leered long time passed lewd comment gis breath
crossing roadsit raining quite heavily man leered long time passed lewd comment gis breath
crossing roadsit raining quite heavily man leered long time passed lewd comment gis breath
late evening around 730 pm shopping market south extn locality delhi trying look auto go passing indecent comments
lady walking makina wearing short dress men started mock without even sympathy saying walk naked tather thatn wearing short dress showing people things
car man urinating open ground near lyon maid company acually care whether looking used middle finger send abussive message
car man urinating open ground near lyon maid company acually care whether looking used middle finger send abussive message
lady walking makina wearing short dress men started mock without even sympathy saying walk n

In [ ]:
print(y_test)

[[[0. 1. 0.]
  [0. 1. 0.]
  [1. 0. 0.]]

 [[0. 1. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]

 [[0. 1. 0.]
  [1. 0. 0.]
  [0. 1. 0.]]

 ...

 [[1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]

 [[1. 0. 0.]
  [0. 1. 0.]
  [1. 0. 0.]]

 [[1. 0. 0.]
  [1. 0. 0.]
  [1. 0. 0.]]]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Define the vocabulary size based on the tokenizer word index
vocab_size = len(tokenizer.word_index) + 1

# Define the model
model = Sequential()

# Add an Embedding layer
embedding_dim = 50  # You can adjust the embedding dimension based on your data
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=100))

# Add an LSTM layer
model.add(LSTM(units=100, activation='relu'))
model.add(Dense(units=64, activation='relu'))
# Add Dropout for regularization
model.add(Dropout(0.5))

# Add a Dense layer for each output with softmax activation for multi-class classification
model.add(Dense(units=3, activation='softmax'))  # Output for Commenting, Ogling, Touching

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()
y_train = y_train.reshape(-1, 3)
y_test = y_test.reshape(-1, 3)
print(y_test)

history = model.fit(X_train, y_train,
                    epochs=10, batch_size=64, validation_split=0.2)


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 50)           127700    
                                                                 
 lstm_5 (LSTM)               (None, 100)               60400     
                                                                 
 dense_8 (Dense)             (None, 64)                6464      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 3)                 195       
                                                                 
Total params: 194759 (760.78 KB)
Trainable params: 194759 (760.78 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
[[0. 1. 0.]
 [

KeyboardInterrupt: 

In [ ]:
# https://www.linkedin.com/pulse/hamming-score-multi-label-classification-chandra-sharat/

'''Hamming score for multi label classification'''
def hamming_score(y_true, y_pred):
    acc_list = []

    for i in range(y_true.shape[0]): # for each value in y_true
        set_true = set(np.where(y_true[i])[0])
        set_pred = set(np.where(y_pred[i])[0])

        temp_a = None

        if len(set_true)==0 and len(set_pred) ==0:
            temp_a =1
        else:
            temp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)))
        acc_list.append(temp_a)
    return np.mean(acc_list)


def deep_modeling_report(model, train_data, train_label, test_data, test_label, title="Deep Learning Model"):
    result = {} # dictionary to store all the result of particular model
    print("################ Modeling Report ###################")
    ################ TRAIN ERROR and ACCURACY ###################
    # prediction of test dataset using deep learning models
    test_predicted = (model.predict(test_data)>0.5).astype("int32")
    train_predicted = (model.predict(train_data)>0.5).astype("int32")

    train_accuracy = accuracy_score(train_label, train_predicted)
    test_accuracy = accuracy_score(test_label, test_predicted)

    result['train_accuracy']=train_accuracy
    result['test_accuracy']=test_accuracy

    train_f1_score = f1_score(train_label, train_predicted, average='macro')
    test_f1_score = f1_score(test_label, test_predicted, average='macro')

    result['train_f1_score']=train_f1_score
    result['test_f1_score']=test_f1_score

    train_recall_score = recall_score(train_label, train_predicted, average='macro')
    test_recall_score = recall_score(test_label, test_predicted, average='macro')

    result['train_recall']=train_recall_score
    result['test_recall']=test_recall_score

    train_hamming_loss = hamming_loss(train_label, train_predicted)
    test_hamming_loss = hamming_loss(test_label, test_predicted)
    result['train_hamming_loss'] = train_hamming_loss
    result['test_hamming_loss'] = test_hamming_loss

    train_hamming_score = hamming_score(train_label, train_predicted)
    test_hamming_score = hamming_score(test_label, test_predicted)
    result['train_hamming_score'] = train_hamming_score
    result['test_hamming_score'] = test_hamming_score

    print("TRAIN Accuracy(exact match) : ",train_accuracy)
    print("TEST Accuracy(exact match) : ",test_accuracy)
    print("="*50)
    print("TRAIN f1-score(macro) : ",train_f1_score)
    print("TEST f1-score(macro) : ",test_f1_score)
    print("="*50)
    print("TRAIN hamming loss : ",train_hamming_loss)
    print("TEST hamming loss : ",test_hamming_loss)
    print("="*50)
    print("TRAIN hamming score : ", train_hamming_score)
    print("TEST hamming score : ", test_hamming_score)

    print("#"*50)
    print("Confusion MATRIX")
    print("TRAIN : ",multilabel_confusion_matrix(train_label, train_predicted))
    print("TEST : ", multilabel_confusion_matrix(test_label, test_predicted))
    ###########################################################################

    ############### CLASSIFICATIO RESULT of both Train, and Test dataset ######
    train_cf_report = classification_report(train_label, train_predicted)
    test_cf_report = classification_report(test_label, test_predicted)

    print("Classification report of TRAIN : ")
    print(train_cf_report)
    print("Classification report of TEST : ")
    print(test_cf_report)
    #############################################################################################

    ################### ROC-AUC Score ###########################################################
    # getting train_score, and test_score
    test_prob = model.predict(test_data)
    train_prob = model.predict(train_data)

    # area under the curve
    train_auc = roc_auc_score(train_label, train_prob, average='macro')
    test_auc = roc_auc_score(test_label, test_prob, average='macro')

    result['train_auc']=train_auc
    result['test_auc']=test_auc
    print("#"*50)
    print("TRAIN AUC : ",train_auc)
    print("TEST AUC : ",test_auc)

    ##########  TRAIN AUC  ###########

    # https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#plot-roc-curves-for-the-multilabel-problem

    n_classes = train_label.shape[1] # no.of classes

    # Compute ROC Curve and ROC area for each class
    fpr_train, fpr_test = dict(), dict()
    tpr_train, tpr_test = dict(), dict()
    roc_auc_train, roc_auc_test = dict(), dict()

    for i in range(n_classes):
        fpr_train[i], tpr_train[i], _ = roc_curve(train_label[:,i], train_prob[:,i])
        fpr_test[i], tpr_test[i], _ = roc_curve(test_label[:,i], test_prob[:,i])
        roc_auc_train[i] = auc(fpr_train[i], tpr_train[i])
        roc_auc_test[i] = auc(fpr_test[i], tpr_test[i])

    plt.figure(figsize=(10,7))
    for i in range(n_classes):
        plt.plot(fpr_train[i],tpr_train[i],label='TRAIN AUC(macro) class={} Score={}'.format(i,roc_auc_train[i]))
    plt.plot([0,1],[0,1], label='No skill',color='navy',lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title("TRAIN AUC scores per class of {}".format(title))
    plt.legend(
    plt.show()

    plt.figure(figsize=(10,7))
    for i in range(n_classes):
        plt.plot(fpr_test[i],tpr_test[i],label='TEST AUC(macro) calss={} Score={}'.format(i,roc_auc_test[i]))
    plt.plot([0,1],[0,1], label='No skill',color='navy',lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title("TEST AUC scores per class of {}".format(title))
    plt.legend()
    plt.show()

    result['model'] = title
    return result

# https://github.com/marcotcr/lime/issues/374#issuecomment-524801935
from lime.lime_text import LimeTextExplainer
def deep_multi_label_explainer(model, input_text, labels, num_features=None, num_samples=None, bow=True):
    for label in labels:
        class_names = ['Non-{}'.format(label), label]

        def make_classifier_pipeline(label=label):
            label_index = labels.index(label)
            def lime_explainer_pipeline(texts):
                input_text = pd.Series(texts)# typecasting to series from raw text
                input_sequence = tokenizer.texts_to_sequences(input_text) #changing text into sequences
                padded_input_sequence = pad_sequences(input_sequence, maxlen=maxlen, padding='post') # padding the sequence
                predicted_prob = model.predict(padded_input_sequence)
                prob_true = predicted_prob[:, label_index]
                result = np.transpose(np.vstack(([1-prob_true, prob_true])))
                result  = result.reshape(-1, 2)
                return result

            return lime_explainer_pipeline
        classifier_fn = make_classifier_pipeline(label=label)
        explainer = LimeTextExplainer(
                                        class_names=class_names
                                        kernel_width=25,
                                        bow=bow
        )
        exp = explainer.explain_instance(
            input_text,
            classifier_fn,

        exp.show_in_notebook(text=True, predict_proba=True)